## Задание 12. — Градиентные методы для выпуклой оптимизации

* Реализовать градиентный спуск для нахождения минимума сильно выпуклой функции (одной переменной) (по одной из стратегий выбора шага) и метод Нестерова.
* Сравнить количества итераций.

## Решение:

In [1]:
import random
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

Реализуем градиентный спуск (с постоянным шагом и с дроблением шага):

In [2]:
def gradient_descent(f, grad, x0, delta, eps, eps_del=1):
    '''
        Функция, реализующая градиентный спуск
        
        Parameters
        ----------
        f : function
            сильно выпуклая функция одной переменной, минимум которой ищется
        grad : function
            градиент функции f
        x0: float
            начальное приближение
        delta: float
            требуемая точность
        eps: float
            шаг (первоначальный) градиентного спуска
        eps_del: float
            значение дробления шага эпсилон
            (по умолчанию, 1 - градиентный спуск с постоянным шагом,
             иначе - градиентный спуск с дроблением шага)

        Returns
        ----------
        x0 : float
           точка, в которой функция достигает минимум
        temp_iter: int
            количество итераций
    '''
    prev_x = x0
    x0 = prev_x - eps*grad(prev_x)
    
    temp_iter = 1
    while np.linalg.norm(x0 - prev_x) > delta:
        prev_x = x0
        x0 = prev_x - (eps/(eps_del**temp_iter))*grad(prev_x)
        temp_iter += 1
        
    return x0, temp_iter

Реализуем метод Нестерова:

In [3]:
def nesterov_method(f, grad, y, eps=1e-6):
    '''
        Функция, реализующая градиентный спуск методом Нестерова
        
        Parameters
        ----------
        f : function
            сильно выпуклая функция одной переменной, минимум которой ищется
        grad : function
            градиент функции f
        y0: float
            начальное приближение

        Returns
        ----------
        x0 : float
           точка, в которой функция достигает минимум
        temp_iter: int
            количество итераций
    '''
    # Значения на нулевой итерации
    a = 1
    x = y
    alpha = np.linalg.norm(y - 1.5*y) / np.linalg.norm(grad(y) - grad(1.5*y))
    
    temp_iter = 1
    while abs(grad(x)) > eps:
        i = 0
        while f(y) - f(y - (2 ** (-i))*alpha*grad(y)) < (2 ** (-i-1)) * alpha * (np.linalg.norm(y) ** 2):
            i += 1
        
        alpha *= 2**(-i)
        
        prev_x = x
        x = y - alpha*grad(y)
        
        prev_a = a
        a = (1 + np.sqrt(4*(a**2) + 1))/2
        
        y = x + (prev_a - 1)*(x - prev_x) / a
        
        temp_iter += 1
        
    return x, temp_iter

Зададим функцию тестирования градиентного спуска:

In [7]:
def testing_gradient(
    f = lambda x: x ** 2 - 6*x + 42, 
    grad = lambda x: 2*x - 6, 
    solution = 3,
    x0 = 4.75, 
    delta = 1e-6, 
    eps = [0.5, 0.1, 0.05, 0.01],
    eps_del = [1, 1.1, 1.2],
    need_plots = True
):
    '''
        Тестирование градиентного спуска
        
        Parameters
        ----------
        f : function
            сильно выпуклая функция одной переменной, минимум которой ищется
        grad : function
            градиент функции f
        solution: float
            аналитически найденная точка минимума
        x0: float
            начальное приближение
        delta: float
            требуемая точность
        eps: list
            массив шагов (первоначальных) градиентного спуска
        eps_del: list
            массив значений дробления шага эпсилон
        need_plot: bool
            необходим ли график

        Returns
        ----------
        df : pd.DataFrame
            датафрейм с нормой разности найденных точек и решения, 
            количеством итераций, шагом эпсилон и стратегией выбора шага
    '''
    res = []
    for _eps in eps:
        for _del in eps_del:
            temp = gradient_descent(f, grad, x0, delta, _eps, _del)
            
            res.append({
                'diff':          abs(solution - temp[0]),
                'iter_count':    temp[1],
                'eps':           _eps,
                'eps_delimiter': _del
            })
    
    df = pd.DataFrame({
        'diff':          [x['diff'] for x in res],
        'iter_count':    [x['iter_count'] for x in res],
        'eps':           [x['eps'] for x in res],
        'eps_delimiter': [x['eps_delimiter'] for x in res]
    })
    
    if need_plots:
        fig = px.line(df, x='diff', y='iter_count', 
                      color='eps', markers=True,
                      labels={
                          'eps': 'Скорость обучения',  
                          'iter_count': 'Количество итераций', 
                          'diff': 'Погрешность'
                      })
        fig.show()
    
    return df

In [8]:
testing_gradient()

,diff,iter_count,eps,eps_delimiter
0,0.000000,2,0.50,1.0
1,0.000000,2,0.50,1.1
2,0.000000,2,0.50,1.2
3,0.000003,59,0.10,1.0
4,0.170717,111,0.10,1.1
5,0.490143,65,0.10,1.2
6,0.000009,116,0.05,1.0
7,0.565185,116,0.05,1.1
8,0.944041,64,0.05,1.2
9,0.000049,519,0.01,1.0


Применим к нашей функции метод Нестерова: 

In [6]:
res = nesterov_method(
    f = lambda x: x ** 2 - 6*x + 42, 
    grad = lambda x: 2*x - 6, 
    y = 2
)

print(f'Минимум в точке:\t{res[0]}')
print(f'Количество итераций:\t{res[1]}')

Минимум в точке:	3.0
Количество итераций:	2
